In [2]:
from datasets import load_dataset
dataset_tilde = load_dataset("tilde_model", lang1="de", lang2="is")

Downloading:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Using custom data configuration de-is-lang1=de,lang2=is


Downloading:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset tilde_model downloaded and prepared to /home/oddurace/.cache/huggingface/datasets/tilde_model/de-is-lang1=de,lang2=is/0.0.0/cea5250cab9d7f20be33fd2b08330b2e17bf4de020ded2719dda9b288e99a205. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
print(type(dataset_tilde['train']))

<class 'datasets.arrow_dataset.Dataset'>


In [4]:
print(dataset_tilde)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 399681
    })
})


In [5]:
print(dataset_tilde['train'][3]['translation']['de'])

1.1 Zum gegenwärtigen Zeitpunkt wird hart um die Unterstützung der Öffentlichkeit für Islands Antrag auf EU-Mitgliedschaft gerungen, wie die öffentliche Anhörung im September 2010 in Island bestätigte.


In [7]:
!pwd

/mnt/c/Users/oddur/Háskólinn/MalTaekni/Lokaverk/raektin


In [9]:
dataset_tilde['train'].to_parquet('../data/processed_data/tilde_model.parquet')

90436388

In [10]:

dataset_tab = load_dataset('parquet', data_files='../data/processed_data/tilde_model.parquet')

Using custom data configuration default-d3d5525f74025ea3


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /home/oddurace/.cache/huggingface/datasets/parquet/default-d3d5525f74025ea3/0.0.0/9296ce43568b20d72ff8ff8ecbc821a16b68e9b8b7058805ef11f06e035f911a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
print(dataset_tab)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 399681
    })
})


ModuleNotFoundError: No module named 'sanitation'

{'text': '284\tIch spreche kein Japanisch.\t399287\tÉg tala ekki japönsku.'}


In [34]:

import os
from io import open
from datasets import Dataset
from datasets import DatasetDict

def getData():
    ccmatrix_path = os.path.relpath('../data/raw_data/de-is.bitextf.tsv')
    tatoeba_path = os.path.relpath('../data/raw_data/de-is_tatoeba.tsv')

    lines = open(ccmatrix_path, encoding='utf-8').\
            read().strip().split('\n')

    lines2 = open(tatoeba_path, encoding='utf-8').\
            read().strip().split('\n')

    tatoebaDict = { 'id': [], 'translation': [] }

    for i in range(len(lines2)):
        #lines2[i] = re.sub(r"\")
        lines2[i] = lines2[i].split('\t')
        tatoebaDict['id'].append(i)
        tatoebaDict['translation'].append({ 'de': lines2[i][3], 'is': lines2[i][1]})
        
    ccmatrixDict = { 'id': [], 'translation': [] }

    for i in range(len(lines)):
        lines[i] = lines[i].split('\t')
        ccmatrixDict['id'].append(i)
        ccmatrixDict['translation'].append({ 'de': lines[i][2], 'is': lines[i][1] })

    tatoebaDataset = Dataset.from_dict(tatoebaDict)

    ccmatrixDataset = Dataset.from_dict(ccmatrixDict)

    tatoebaData = DatasetDict({'train': tatoebaDataset})

    ccmatrixData = DatasetDict({'train': ccmatrixDataset})

    return tatoebaData, ccmatrixData


In [35]:
import re

tatoebaData, ccmatrixData = getData()

def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-ZÁáÐðÉéÍíÓóÚúÝýÞþÆæÖöÜüÄäß.!?]+", r" ", s)
    return s

def normalizeDataset(dataset):
    pairs = []

    for i in range(len(dataset['train'])):
        german = dataset['train'][i]['translation']['de']
        icelandic = dataset['train'][i]['translation']['is']
        pairs.append([normalizeString(german), normalizeString(icelandic)])
    return pairs

tatoebaPairs = normalizeDataset(tatoebaData)

ccmatrixPairs = normalizeDataset(ccmatrixData)


In [41]:
tatoebaData['train'].to_parquet('../data/processed_data/de-is_tatoeba_raw.parquet')
ccmatrixData['train'].to_parquet('../data/processed_data/de-is_ccmatrix_raw.parquet')

145446406456

In [45]:
testload = load_dataset('parquet', data_files='../data/processed_data/de-is_tatoeba_raw.parquet')
test2load = load_dataset('parquet', data_files='../data/processed_data/de-is_ccmatrix_raw.parquet')

Using custom data configuration default-dacad57c8c5cd75f
Reusing dataset parquet (/home/oddurace/.cache/huggingface/datasets/parquet/default-dacad57c8c5cd75f/0.0.0/9296ce43568b20d72ff8ff8ecbc821a16b68e9b8b7058805ef11f06e035f911a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-f012256bdec0ddc6


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /home/oddurace/.cache/huggingface/datasets/parquet/default-f012256bdec0ddc6/0.0.0/9296ce43568b20d72ff8ff8ecbc821a16b68e9b8b7058805ef11f06e035f911a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [46]:
print(testload)
print(test2load)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 1013
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 3377139
    })
})


In [51]:
print(testload['train'][1]['translation'])

{'de': 'Ég tala ekki japönsku.', 'is': 'Ich spreche kein Japanisch.'}
